In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import yfinance as yf
import sp_components_data as sp
# from fmp_python import company_valuation

import sqlite3


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pricing_date = '2024-08-30' 

## Getting the data

#### S&P constituent price data

In [3]:
# price data
# data = sp.get_sp_data()

# S&P price
# sp500 = yf.Ticker("^GSPC")
# sp500_constituents = sp500.history(period="1d")
# print(sp500_constituents)
with open('sp_components_data.pkl', 'rb') as f:
    data = pickle.load(f)
data.head(3)

Price                      Adj Close                                 \
Ticker                             A        AAL      AAPL ABBV ABNB   
Date                                                                  
2008-01-02 00:00:00+00:00  23.293402  12.557317  5.876340  NaN  NaN   
2008-01-03 00:00:00+00:00  23.062395  11.991672  5.879057  NaN  NaN   
2008-01-04 00:00:00+00:00  22.311621  11.699421  5.430277  NaN  NaN   

Price                                                                 \
Ticker                           ABT      ACGL        ACN       ADBE   
Date                                                                   
2008-01-02 00:00:00+00:00  18.215652  7.608889  26.544682  41.709999   
2008-01-03 00:00:00+00:00  18.104681  7.764444  26.088289  41.790001   
2008-01-04 00:00:00+00:00  18.209129  7.702222  25.698139  40.360001   

Price                                 ...  Volume                             \
Ticker                           ADI  ...     WTW       WY     WYNN      XEL   
Date                                  ...                                      
2008-01-02 00:00:00+00:00  20.164034  ...  329746  1867500  1629800  1752000   
2008-01-03 00:00:00+00:00  19.858608  ...  320422  2158200  2432900  1589400   
2008-01-04 00:00:00+00:00  19.340736  ...  478406  2988500  3068700  2448400   

Price                                                                  
Ticker                          XOM XYL      YUM      ZBH    ZBRA ZTS  
Date                                                                   
2008-01-02 00:00:00+00:00  23351900 NaN  4132244  1476608  720200 NaN  
2008-01-03 00:00:00+00:00  19160600 NaN  3846950  1570441  467600 NaN  
2008-01-04 00:00:00+00:00  24730900 NaN  6056692  2314101  401400 NaN  

[3 rows x 3018 columns]

#### Company Rating

In [36]:
rating_data = pd.read_csv('corporate_rating.csv')
rating_data['Date'] = pd.to_datetime(rating_data['Date'])
date_df = rating_data.groupby(['Symbol']).agg(rating_date = ('Date', np.max))
rating_data = rating_data.merge(date_df, left_on='Symbol', right_on='Symbol')
rating_data = rating_data[rating_data['Date'] == rating_data.rating_date].reset_index()
rating_data.head(3)

,index,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,...,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,rating_date
0,4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,2016-10-24,Consumer Durables,0.957844,0.495432,0.141608,...,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790,2016-10-24
1,7,A,Schlumberger N.V.,SLB,Egan-Jones Ratings Company,2015-07-23,Energy,1.905814,1.544791,0.197791,...,0.726292,5.047356,10.287293,1.908484,0.081212,15.264562,6.949487,0.248203,3.665200,2015-07-23
2,9,A,Honeywell International Inc.,HON,Moody's Investors Service,2016-02-16,Capital Goods,1.091557,0.850961,0.296935,...,0.803264,5.618107,9.692229,2.697369,0.170706,10.362903,6.994101,0.141365,4.793369,2016-02-16


#### S&P stocks ticker data

In [71]:
sector_weights = pd.read_csv('sector_weights.csv', usecols = [1,2])
sector_weights

,Sector,Weight
0,Industrials,0.071126
1,Healthcare,0.113935
2,Technology,0.300976
3,Utilities,0.023360
4,Financial Services,0.127863
5,Basic Materials,0.018510
6,Consumer Cyclical,0.097383
7,Real Estate,0.022239
8,Communication Services,0.129858
9,Consumer Defensive,0.062324


In [34]:
# ticker_data = sp.sp_ticker_data()
with open('sp_ticker_data.pkl', 'rb') as f:
    ticker = pickle.load(f)
ticker.head(3)

,symbol,sector,currentPrice,ebitdaMargins,profitMargins,grossMargins,revenueGrowth,operatingMargins,recommendationKey,earningsGrowth,...,trailingAnnualDividendRate,averageVolume10days,dividendRate,trailingPE,regularMarketVolume,averageVolume,volume,fiveYearAvgDividendYield,dividendYield,trailingPegRatio
0,MMM,Industrials,134.69,0.22536,0.02891,0.44763,-0.004,0.08169,buy,NaN,...,5.21,3277340,2.80,52.408560,6091739,4286588,6091739,4.36,0.0208,NaN
1,AOS,Industrials,83.72,0.21099,0.14674,0.38351,0.066,0.20238,hold,0.019,...,1.26,640560,1.28,21.633076,1007063,916825,1007063,1.74,0.0153,2.0570
2,ABT,Healthcare,113.27,0.25957,0.13645,0.55475,0.040,0.17664,buy,-0.051,...,2.16,3905220,2.20,35.731860,4320046,6347933,4320046,1.63,0.0194,4.5813


In [43]:
full_df = ticker.merge(rating_data, left_on='symbol', right_on='Symbol', how = 'left' )

(503, 89)

In [50]:
no_rating = full_df[full_df.Rating.isnull() == True]
pd.unique(full_df.Rating)

array(['AA', nan, 'BBB', 'A', 'BB', 'B', 'C', 'AAA'], dtype=object)